In [59]:
from pearce.emulator.emu import OriginalRecipe, ExtraCrispy
from pearce.emulator.trainingData import parameter, PARAMS

In [60]:
from george.kernels import ExpSquaredKernel
import george

In [61]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from itertools import cycle
colors = cycle(sns.color_palette())

In [62]:
training_dir = '/u/ki/swmclau2/des/PearceLHC/'

In [63]:
or_params = PARAMS[:]
or_params.append(parameter('r', 0, 1)) #95% sure bounds aren't used for r
#emu = OriginalRecipe(training_dir,or_params)
emu = ExtraCrispy(training_dir)

/u/ki/swmclau2/.conda/envs/hodemulator/lib/python2.7/site-packages/pearce/emulator/emu.py:695: UserWarning: WARNING: NaN detected. Skipping point in /u/ki/swmclau2/des/PearceLHC/cov_HOD133.npy
  warnings.warn('WARNING: NaN detected. Skipping point in %s' % cov_file)


In [64]:
ig = emu.get_initial_guess('xi', {})
a = ig['amp']
kernel = a * ExpSquaredKernel(emu.metric, ndim=emu.emulator_ndim)
loo_gp = george.GP(kernel)
# gp = george.GP(kernel, solver=george.HODLRSolver, nleaf=x.shape[0]+1,tol=1e-18)

loo_gp.compute(emu.x[1:], emu.yerr[1:])  # NOTE I'm using a modified version of george!

In [65]:
K_inv_full = emu.gp.solver.apply_inverse(np.eye(emu.x.shape[0]),in_place=True)

In [66]:
y_param = 'logMmin'
emulation_point = [('f_c', 0.233), ('logM0', 12.0), ('sigma_logM', 0.533), 
                   ('alpha', 1.083),('logM1', 13.5), ('logMmin', 12.233)]
em_params = dict(emulation_point)
#del em_params[y_param]

In [67]:
from os import path
save_dir = '/u/ki/swmclau2/des/EmulatorMCMC/'
rpoints = np.log10(np.loadtxt(path.join(save_dir, 'rpoints.npy')) )
true_vals = np.log10( np.loadtxt(path.join(save_dir, 'xi.npy')) )

In [68]:
rbins = np.array([  0.06309573,   0.12437607,   0.24517359,   0.34422476, 0.48329302, 0.67854546,\
           0.9526807 , 1.33756775,1.8779508 ,   2.6366509 ,   3.70186906,   5.19743987, 7.29722764, \
           10.24533859,  14.38449888,  20.1958975 , 28.35512583,  39.81071706] )
#rpoints= np.log10((rbins[1:]+rbins[:-1])/2)
#em_params['r'] = rpoints#[:2]

In [69]:
input_params = {}
input_params.update(em_params)

# i'd like to remove 'r'. possibly requiring a passed in param?
t_list = [input_params[p.name] for p in emu.ordered_params]
t_grid = np.meshgrid(*t_list)
t = np.stack(t_grid).T
t = t.reshape((-1, emu.emulator_ndim))
try:
    t = np.sort(t.view(','.join(['float64' for _ in xrange(min(t.shape))])),
                order=['f%d' % i for i in xrange(min(t.shape))], axis=0).view(np.float)
except ValueError:
        t = np.sort(t.view(','.join(['float64' for _ in xrange(max(t.shape))])),
                order=['f%d' % i for i in xrange(max(t.shape))], axis=0).view(np.float)

In [70]:
print t.shape

(1, 6)


In [73]:
print emu.y.shape

(496, 17)


In [74]:
loo_mu = loo_gp.predict(emu.y[1:, 0], t , mean_only=True)
#loo_mu = loo_gp.predict(emu.y[1:], t , mean_only=True)
print loo_mu

[ 3.33449327]


In [75]:
loo_inv = loo_gp.solver.apply_inverse(np.eye(emu.x.shape[0]-1),in_place=True)
print loo_inv.shape, K_inv_full.shape

(495, 495) (496, 496)


In [76]:
print loo_inv[0,:]
print 
print loo_inv[1,:]

[  1.16844815e+02  -2.75080377e+01  -3.57517930e+01  -2.20421231e+01
  -1.95319000e+01   3.53334317e+00   2.57459050e+00   3.05744799e+00
  -1.08340738e+01   4.65554592e+00   5.52501535e+00   5.15286791e+00
   5.89697914e+00   5.49312570e+00  -1.94877792e+00   8.33244083e-01
  -1.20853715e+01  -4.35705006e+00   3.35757580e+00  -4.11220263e+01
  -2.17695051e+00  -2.82288053e+01   7.84846814e+00   5.05152053e+00
  -1.32456335e+01   1.01231081e+01  -7.91824871e+01  -6.88145011e+00
  -3.18462645e+00   1.28622477e+01  -4.98832062e+00  -2.28235072e+00
   6.98063769e+00   1.34345614e+01   3.54944927e+01   5.38637378e+00
  -7.05794570e+00  -2.76281656e+00   5.54182914e+00   3.74641098e+00
  -2.55323172e+00   4.16284163e+00   1.07776165e+01  -2.76727479e+00
   6.90135958e+00   1.04418304e+01  -7.90148470e-01  -2.48185019e-01
   2.26775966e+00  -1.51479273e+01   1.90539240e+01   1.32986336e+00
   1.35803619e+00   4.13741725e-02   2.65926109e+01  -9.17798792e+01
   1.20282148e+01   8.42513405e+00

In [77]:
print K_inv_full[0,:]
print 
print K_inv_full[1,:]

[  9.05074510e+01  -3.01298137e+00  -1.35212310e+01   6.27946377e+00
  -2.96631762e+00  -4.38134673e+00   1.41161020e+00   3.24304425e+00
  -3.43288019e-01   4.05925946e-01   5.88197600e+00   8.29678061e+00
   5.79767528e+00   4.44535786e+00  -8.08546001e+00   7.40174336e-01
  -3.01131598e+00  -1.62179550e+00  -2.59864540e-01   2.58835066e+00
  -8.25337272e-01  -3.44111463e+00   3.15075436e+00  -5.72698666e+00
   1.11893363e+00  -6.21572511e+00  -1.71776960e+00   8.91132915e-01
  -1.76822913e+00   2.05262269e-02   2.11856190e+00  -3.45268006e-01
  -8.15299769e-01   2.94706005e+00  -1.94831637e+00  -3.18231501e+00
  -2.80385768e+00   3.97287749e-02  -5.37362555e+00  -2.52861969e+00
   1.98942986e+00  -2.79445457e+00   1.30597478e+00   3.26641410e+00
   3.92634458e-01  -2.80989283e+00  -1.41525758e+00  -1.25381347e+00
   3.79432095e+00  -4.75642316e+00   8.86654122e+00   1.78038080e+01
  -1.28196047e+00   9.09404965e-01  -1.79546318e+00   4.93430035e+00
  -8.01317571e+00   2.34885988e-01

In [80]:
mu, G = emu.emulate(em_params)
print mu

[ 3.33857219  2.93358166  2.48636433  2.21810416  1.86595312  1.48862137
  1.09563825  0.75276058  0.48297387  0.25980587  0.02334519 -0.20475356
 -0.47153732 -0.73603583 -1.04181694 -1.39814147 -1.95044229]


In [81]:
#yhat = loo_gp.predict(emu.y[1:], emu.x[0].reshape((1,emu.x.shape[1])), mean_only=True)
yhat = loo_gp.predict(emu.y[1:, 0], emu.x[0].reshape((1,emu.x.shape[1])), mean_only=True)

print yhat

[ 3.91558462]


In [82]:
#print emu.y[0, 0]
print emu.y[0]

[ 3.72177668  3.14605283  2.82388645  2.45748763  2.07921166  1.6411642
  1.22416032  0.88683411  0.62006108  0.39436861  0.15029365 -0.05740708
 -0.32774724 -0.60372225 -0.91447297 -1.2749174  -1.76488427]


In [83]:
x, y = emu.x[:], emu.y[:]

N = K_inv_full.shape[0]

In [84]:
from time import time
from scipy.linalg import inv

In [85]:
my_inds = [i for i in xrange(1, N-1)]
my_inds.append(0)
my_inds = np.array(my_inds)

In [86]:
from copy import deepcopy

In [87]:
times = np.zeros((N,))

In [88]:
mus = np.zeros((N, len(rpoints)))

In [91]:
for idx in xrange(N):
    print idx
    t0 = time()
    #TODO put switches at the end of the loop too
    #then i can make just one deepcopy
    #t_x = deepcopy(emu.x)
    t_x = emu.x
    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    
    #t_yerr = deepcopy(emu.yerr)
    t_yerr = emu.yerr
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    
    #t_y = deepcopy(emu.y[:, 0])
    t_y = emu.y
    t_y[[N-1, idx], :] = t_y[[idx, N-1], :]
    #t_y[[N-1, idx]] = t_y[[idx, N-1]]
    #K_inv = deepcopy(K_inv_full)
    K_inv = K_inv_full
    
    K_inv[[idx, N - 1], :] = K_inv[[N - 1, idx], :]
    K_inv[:, [idx, N - 1]] = K_inv[:, [N - 1, idx]]
    
    K_m_idx_inv = K_inv[:N - 1, :][:,:N - 1] \
            -np.outer(K_inv[N-1,:N-1], K_inv[:N-1,N-1])/K_inv[N-1,N-1]
           
    alpha_m_idx = [np.dot(K_m_idx_inv, t_y[:N-1, rbin ]-emu.gp.mean(t_x[:N-1])) for rbin in xrange(t_y.shape[1]) ]
    #alpha_m_idx = np.dot(K_m_idx_inv, t_y[:N-1 ]-emu.gp.mean(t_x[:N-1]))

    Kxxs_t = emu.gp.kernel.value(t, t_x[:N-1])
    mus[idx, :] = np.array([np.dot(Kxxs_t, alpha_m_idx[rbin])+ emu.gp.mean(t) for rbin in xrange(t_y.shape[1]) ])[:,0]
    #mus[idx, :] = np.dot(Kxxs_t, alpha_m_idx)+ emu.gp.mean(t)

    #print mus[idx]
    print time()-t0, 's'
    #print
    K_inv_full[[idx, N - 1], :] = K_inv_full[[N - 1, idx], :]
    K_inv_full[:, [idx, N - 1]] = K_inv_full[:, [N - 1, idx]]

    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    t_y[[N-1, idx],:] = t_y[[idx, N-1],:]
    #t_y[[N-1, idx]] = t_y[[idx, N-1]]
    
    

0
0.0151062011719 s
1
0.0119590759277 s
2
0.0117771625519 s
3
0.0118639469147 s
4
0.0119609832764 s
5
0.0118670463562 s
6
0.0119140148163 s
7
0.0118379592896 s
8
0.0119369029999 s
9
0.0118720531464 s
10
0.0119359493256 s
11
0.0118210315704 s
12
0.0119318962097 s
13
0.0118610858917 s
14
0.0120689868927 s
15
0.0118570327759 s
16
0.0119760036469 s
17
0.0117900371552 s
18
0.0120379924774 s
19
0.0118818283081 s
20
0.0119180679321 s
21
0.0118050575256 s
22
0.011873960495 s
23
0.011932849884 s
24
0.0119090080261 s
25
0.0119099617004 s
26
0.0119359493256 s
27
0.0118000507355 s
28
0.0120429992676 s
29
0.0119090080261 s
30
0.0119261741638 s
31
0.011932849884 s
32
0.0118799209595 s
33
0.0118381977081 s
34
0.0119428634644 s
35
0.0119159221649 s
36
0.0118668079376 s
37
0.011862039566 s
38
0.0119111537933 s
39
0.0119280815125 s
40
0.0118470191956 s
41
0.0118432044983 s
42
0.0118751525879 s
43
0.011824131012 s
44
0.0119700431824 s
45
0.0119152069092 s
46
0.0119290351868 s
47
0.0118300914764 s
48
0.01

In [97]:
for idx in xrange(N):
    print idx
    t0 = time()
    #TODO put switches at the end of the loop too
    #then i can make just one deepcopy
    #t_x = deepcopy(emu.x)
    t_x = emu.x
    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    
    #t_yerr = deepcopy(emu.yerr)
    t_yerr = emu.yerr
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    
    #t_y = deepcopy(emu.y[:, 0])
    t_y = emu.y
    t_y[[N-1, idx], :] = t_y[[idx, N-1], :]
    #t_y[[N-1, idx]] = t_y[[idx, N-1]]
    loo_gp.compute(t_x[:N-1], t_yerr[:N-1]) 
    mus[idx,:] = np.array([loo_gp.predict(emu.y[:N-1, rbin], t , mean_only=True) for rbin in xrange(t_y.shape[1]) ])[:,0]

    print time()-t0, 's'
    #print
    K_inv_full[[idx, N - 1], :] = K_inv_full[[N - 1, idx], :]
    K_inv_full[:, [idx, N - 1]] = K_inv_full[:, [N - 1, idx]]

    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    t_y[[N-1, idx],:] = t_y[[idx, N-1],:]
    #t_y[[N-1, idx]] = t_y[[idx, N-1]]

0
0.0476880073547 s
1
0.0302619934082 s
2
0.0355820655823 s
3
0.0308799743652 s
4
0.0369849205017 s
5
0.0325660705566 s
6
0.0288350582123 s
7
0.0298478603363 s
8
0.0290060043335 s
9
0.0290839672089 s
10
0.0289289951324 s
11
0.0288760662079 s
12
0.0295929908752 s
13
0.0287129878998 s
14
0.0287909507751 s
15
0.0468990802765 s
16
0.0307359695435 s
17
0.031898021698 s
18
0.0337810516357 s
19
0.0300898551941 s
20
0.0297839641571 s
21
0.0290510654449 s
22
0.0279350280762 s
23
0.0282890796661 s
24
0.0313391685486 s
25
0.0329508781433 s
26
0.0280520915985 s
27
0.0278520584106 s
28
0.0315251350403 s
29
0.0276520252228 s
30
0.0284969806671 s
31
0.0277500152588 s
32
0.0280649662018 s
33
0.0409281253815 s
34
0.0276920795441 s
35
0.0275008678436 s
36
0.0297679901123 s
37
0.0296709537506 s
38
0.0274200439453 s
39
0.0276811122894 s
40
0.0281209945679 s
41
0.0314960479736 s
42
0.0307660102844 s
43
0.0280480384827 s
44
0.0274460315704 s
45
0.027370929718 s
46
0.0298039913177 s
47
0.0299429893494 s
48
0

In [ ]:
#print mus[:, 0]

In [ ]:
(N-1)/float(N)

In [ ]:
(N-1.0)/N*np.cov(mus, rowvar=False)

In [ ]:
print G